# Example of using LiteQTL

This notebook showcase how to use LiteQTL to perform a QTL scan.

### Dataset: 
We use the BXD Genotype Database (GN Accession: GN600) and a sets of transcriptome data, UTHSCAffy MoGene 1.0 ST Spleen (GN Accession: GN283) from GeneNetwork. 

The genotype file includes 7321 markers by 198 BXD strains; 

The spleen dataset has data for 79 BXD strains and for 35556 transcripts. 

Data cleaning and wrangling was performed using R/qtl (Broman et al. 2003) and R/qtl2 (Broman et al. 2019). 

For details regarding data cleaning, please see [this script](https://github.com/senresearch/LiteQTL-G3-supplement/blob/master/code/clean-data.R)

In [1]:
using Revise
using Pkg
Pkg.activate("/export/xiaoqihu/git/LiteQTL.jl/")

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /home/xiaoqihu/.julia/packages/Revise/moD4B/src/packagedef.jl:1361
 Activating environment at `/export/xiaoqihu/git/LiteQTL.jl/Project.toml`


In [2]:
# Loading packages
using LiteQTL
using DelimitedFiles
using BenchmarkTools

In [3]:
using Pkg 
Pkg.status("LiteQTL")

Project LiteQTL v0.1.2
No Matches in `/export/xiaoqihu/git/LiteQTL.jl/Project.toml`


In [4]:
# Location of genotype file and phenotype file
geno_file = joinpath(@__DIR__, "..", "data", "processed", "spleen-bxd-genoprob.csv")
pheno_file = joinpath(@__DIR__, "..", "data","processed", "spleen-pheno-nomissing.csv")

# Whether to output whole LOD matrix or only maximum LOD of each phenptype? Default is false (only maximum)
export_matrix = false

# Location of output 
output_file = "output.csv"

# Setting BLAS threads as 16 threads. 
LiteQTL.set_blas_threads(16);

# Read in data.
G = LiteQTL.get_geno_data(geno_file, Float64)
Y = LiteQTL.get_pheno_data(pheno_file, Float64, transposed=false)
# getting geno and pheno file size.
n = size(Y,1) # number of individuals
m = size(Y,2) # number of Traits
p = size(G,2) # number of Markers
println("******* Indivuduals n: $n, Traits m: $m, Markers p: $p ****************");

Number of threads using: 16


┌ Info: Removing sex column of phenotype. 
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/data_io.jl:29


******* Indivuduals n: 79, Traits m: 35554, Markers p: 7321 ****************


In [6]:
# run genome scan. 
lod = LiteQTL.scan(Y, G,n;export_matrix);
# @benchmark LiteQTL.scan(Y, G,n;export_matrix)

┌ Info: Running genome scan...
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/cpu.jl:91
┌ Info: Done calculating corelation coefficients.
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/cpu.jl:96


Calculating max lod


┌ Info: Done calculating LOD. 
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/cpu.jl:100


In [8]:
# Uncomment the following lines to run the same function but with GPU: 
# lod = LiteQTL.scan(Y, G, n; usegpu=true)
# @benchmark LiteQTL.scan(Y, G, n; usegpu=true)

35554×2 Array{Float64,2}:
 5046.0   2.03595
 3111.0   2.74552
 6631.0   2.38099
 6632.0   3.03683
 5326.0   2.6874
 5326.0   2.93972
 5325.0   3.40571
 3120.0   2.73382
 3214.0   1.93849
 6631.0   2.45835
 3111.0   2.96902
 5326.0   2.58704
 5326.0   1.90628
    ⋮    
 2975.0   1.69443
 3111.0   4.3085
 2110.0  15.6794
 7255.0   1.97968
 1808.0   2.48822
 6546.0   2.85884
 2110.0   3.24836
 3717.0   2.21924
 7136.0   2.22245
 6647.0   4.70312
 3085.0   2.76619
 3549.0   1.7484

In [10]:
# run genome scan with covariates. 
# X is covariates
# X = ones(size(pheno)[1]) # standard covariates

# Use the first column of phenotype as covariates. 
X = reshape(Y[:,1], :,1) # make 1 column as a 2d matrix
Y = Y[:,2:end]

@time lod = LiteQTL.scan(Y, G, X, n; export_matrix)

┌ Info: Running genome scan with covariates...
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/cpu.jl:44


Calculating max lod
 12.552027 seconds (4.18 M allocations: 2.189 GiB, 6.16% gc time)


35552×2 Array{Float64,2}:
 6631.0   2.22609
 6633.0   2.93644
 5326.0   2.70782
 5326.0   2.4973
 5325.0   3.79566
 5325.0   1.91135
 5405.0   2.20111
 6631.0   2.49729
 2107.0   2.10042
 5326.0   2.37707
 5326.0   2.02948
 5326.0   2.0498
 5326.0   1.98242
    ⋮    
 5758.0   1.90361
 7315.0   2.62971
 2110.0  18.2555
 7255.0   1.99085
 1808.0   2.43012
 6547.0   2.69031
 2110.0   3.2642
 3719.0   3.03524
 6981.0   2.45788
 6647.0   4.58176
 6621.0   2.24316
 3057.0   1.95288

In [ ]:
# write output to file
writedlm(joinpath(Base.@__DIR__, "..", "data", "results", output_file), lod, ',')